In [ ]:
import os
import json
import re
import html
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler

# Functions and Dictionaries

In [2]:

def extract_country_code(filepath):
    basename = os.path.basename(filepath)       # e.g. "hu.json"
    code, _ = os.path.splitext(basename)         # e.g. "hu"
    if re.match(r'^[a-zA-Z]{2}$', code):
        return code.upper()
    else:
        return code[:2].upper()


In [ ]:
# Use regex to parse factbook "religion" strings
RELIGION_GROUPS = {
    "Protestant":      r"(?i)\b(calvinist|protestant|evangelical|presbyterian|methodist|congregational|assemblies of god|adventist|brethren|baptist|lutheran)\b",
    "Catholic":        r"(?i)\b(catholic|roman catholic)\b",
    "Orthodox":        r"(?i)\borthodox\b",
    "Muslim":          r"(?i)\b(muslim|islam|sunni|shia)\b",
    "Jewish":          r"(?i)\bjewish|judaism\b",
    "Other Christian": r"(?i)\bchristian\b"
}

def split_outside_parentheses(text: str):
    chunks = []
    current = []
    paren_count = 0
    for char in text:
        if char == '(':
            paren_count += 1
        elif char == ')':
            paren_count -= 1
        if char == ',' and paren_count == 0:
            chunks.append(''.join(current).strip())
            current = []
        else:
            current.append(char)
    if current:
        chunks.append(''.join(current).strip())
    return chunks

def parse_chunk(chunk: str):
    chunk = chunk.strip()
    if "includes" in chunk.lower():
        match = re.search(r'((<\s*1)|(\d+(\.\d+)?))%', chunk)
        if match:
            pct = 1.0 if match.group(2) else float(match.group(3))
        else:
            pct = 0.0
        main_label = chunk.split("(", 1)[0].strip().lower()
        inc_match = re.search(r'includes\s+([^)]+)', chunk, re.IGNORECASE)
        if inc_match:
            sublabels = [s.strip().lower() for s in inc_match.group(1).split(',')]
        else:
            sublabels = []
        return main_label, pct, sublabels
    else:
        regex = r'^(.*?)\s*(?:\([^)]*\)\s*)?((<\s*1)|(\d+(\.\d+)?))%'
        match = re.search(regex, chunk)
        if match:
            label = match.group(1).strip().lower()
            pct = 1.0 if match.group(3) else float(match.group(4))
            return label, pct, None
        else:
            return None, 0.0, None

def parse_religions(text: str):
    text = html.unescape(text)
    results = {key: 0.0 for key in RELIGION_GROUPS}
    results["None"] = 0.0
    results["Other"] = 0.0
    chunks = split_outside_parentheses(text)
    for chunk in chunks:
        label, pct, sublabels = parse_chunk(chunk)
        if label is None:
            continue

        if sublabels is not None:
            if label in ["other", "others"]:
                found = []
                for sub in sublabels:
                    for key, regex in RELIGION_GROUPS.items():
                        if re.search(regex, sub):
                            found.append(key)
                            break
                if found:
                    share = pct / len(found)
                    for f in found:
                        results[f] += share
                else:
                    results["Other"] += pct
            elif "other christian" in label:
                found = []
                for sub in sublabels:
                    for key, regex in RELIGION_GROUPS.items():
                        if re.search(regex, sub):
                            found.append(key)
                            break
                if found:
                    share = pct / len(found)
                    for f in found:
                        results[f] += share
                else:
                    results["Other"] += pct
            else:
                assigned = False
                for key, regex in RELIGION_GROUPS.items():
                    if re.search(regex, label):
                        results[key] += pct / 2
                        assigned = True
                        break
                if not assigned:
                    if label.startswith("none") or label.startswith("no answer"):
                        results["None"] += pct / 2
                    else:
                        results["Other"] += pct / 2
                found = []
                for sub in sublabels:
                    for key, regex in RELIGION_GROUPS.items():
                        if re.search(regex, sub):
                            found.append(key)
                            break
                if found:
                    share = (pct / 2) / len(found)
                    for f in found:
                        results[f] += share
                else:
                    results["Other"] += pct / 2
        else:
            assigned = False
            for key, regex in RELIGION_GROUPS.items():
                if re.search(regex, label):
                    results[key] += pct
                    assigned = True
                    break
            if not assigned:
                if label.startswith("none") or label.startswith("no answer"):
                    results["None"] += pct
                else:
                    results["Other"] += pct
    return results


In [4]:
def process_json_file(filepath):
    with open(filepath, encoding='utf-8') as f:
        data = json.load(f)
    
    country_code = extract_country_code(filepath)
    
    try:
        country = data["Government"]["Country name"]["conventional short form"]["text"]
    except KeyError:
        country = os.path.splitext(os.path.basename(filepath))[0]
    
    try:
        religions = data["People and Society"]["Religions"]["text"]
    except KeyError:
        religions = None

    try:
        birth_rate = data["People and Society"]["Birth rate"]["text"]
    except KeyError:
        birth_rate = None

    try:
        total_fertility_rate = data["People and Society"]["Total fertility rate"]["text"]
    except KeyError:
        total_fertility_rate = None

    try:
        gdp_per_capita = data["Economy"]["Real GDP per capita"]["Real GDP per capita 2023"]["text"]
    except KeyError:
        gdp_per_capita = None

    try:
        literacy_rate = data["People and Society"]["Literacy"]["total population"]["text"]
    except KeyError:
        literacy_rate = None

    try:
        urban_population = data["People and Society"]["Urbanization"]["urban population"]["text"]
    except KeyError:
        urban_population = None

    try:
        population = data["People and Society"]["Population"]["total"]["text"]
    except KeyError:
        population = None

    try:
        median_age = data["People and Society"]["Median age"]["total"]["text"]
    except KeyError:
        median_age = None

    try:
        pop_growth_rate = data["People and Society"]["Population growth rate"]["text"]
    except KeyError:
        pop_growth_rate = None

    if religions:
        religion_dict = parse_religions(religions)
    else:
        religion_dict = {}
    
    record = {
        "Country": country,
        "Country Code": country_code,
        "Religions": religions,
        "Birth rate": birth_rate,
        "Total fertility rate": total_fertility_rate,
        "GDP per capita": gdp_per_capita,
        "Literacy rate": literacy_rate,
        "Urban population": urban_population,
        "Population": population,
        "Median age": median_age,
        "Population growth rate": pop_growth_rate
    }
    record.update(religion_dict)
    return record

In [5]:

def clean_numeric_column(series, pattern=r"([\d\.]+)"):
    return (
        series.astype(str)
        .str.extract(pattern)[0]
        .str.replace(",", "", regex=True)
        .astype(float)
    )


# Processing Code

We read in all our data from the CIA factbook, which can be download from [Github here](https://github.com/factbook/factbook.json).

In [ ]:

records = []
base_dir = "factbook.json" 

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            filepath = os.path.join(root, file)
            try:
                record = process_json_file(filepath)
                records.append(record)
            except Exception as e:
                print(f"Error processing {filepath}: {e}")

df = pd.DataFrame(records)

In [7]:

df["Birth rate"] = clean_numeric_column(df["Birth rate"])
df["Total fertility rate"] = clean_numeric_column(df["Total fertility rate"])
df["GDP per capita"] = (
    df["GDP per capita"]
    .astype(str)
    .str.extract(r"([\d,]+)")[0]
    .str.replace(",", "", regex=True)
    .astype(float)
)
df["Literacy rate"] = clean_numeric_column(df["Literacy rate"])
df["Urban population"] = clean_numeric_column(df["Urban population"])
df["Population"] = clean_numeric_column(df["Population"], pattern=r"([\d,]+)")
df["Median age"] = clean_numeric_column(df["Median age"])
df["Population growth rate"] = clean_numeric_column(df["Population growth rate"])

essential_cols = ["Birth rate", "Total fertility rate", "GDP per capita", "Literacy rate", "Urban population", "Population", "Median age", "Population growth rate"]
df = df.dropna(subset=essential_cols)
df = df[
    (df["GDP per capita"] > 0) &
    (df["Literacy rate"].between(0, 100)) &
    (df["Urban population"].between(0, 100)) &
    (df["Population"] > 0) &
    (df["Median age"] > 0) &
    (df["Population growth rate"].between(-50, 50))
].reset_index(drop=True)

cols = ['Country', 'Country Code', 'Religions', 'Birth rate', 'Total fertility rate', 'GDP per capita',
        'Literacy rate', 'Urban population', 'Population', 'Median age', 'Population growth rate'] + \
       [col for col in df.columns if col not in ['Country', 'Country Code', 'Religions', 'Birth rate', 'Total fertility rate', 'GDP per capita', 'Literacy rate', 'Urban population', 'Population', 'Median age', 'Population growth rate']]
df = df[cols]


df.head()


,Country,Country Code,Religions,Birth rate,Total fertility rate,GDP per capita,Literacy rate,Urban population,Population,Median age,Population growth rate,Protestant,Catholic,Orthodox,Muslim,Jewish,Other Christian,None,Other
0,xx,XX,"Christian 31.1%, Muslim 24.9%, Hindu 15.2%, Bu...",17.0,2.42,20700.0,86.7,57.5,8.057236e+09,31.0,1.03,0.0,0.0,0.0,24.9,1.0,31.1,0.0,44.00
1,Sri Lanka,CE,"Buddhist (official) 70.2%, Hindu 12.6%, Muslim...",14.5,2.13,13000.0,92.3,19.2,2.198261e+07,34.1,0.39,0.0,6.1,0.0,9.7,0.0,1.3,0.0,82.85
2,India,IN,"Hindu 79.8%, Muslim 14.2%, Christian 2.3%, Sik...",16.2,2.03,9200.0,74.4,36.4,1.409128e+09,29.8,0.72,0.0,0.0,0.0,14.2,0.0,2.3,0.0,83.50
3,Nepal,NP,"Hindu 81.2%, Buddhist 8.2%, Muslim 5.1%, Kirat...",17.0,1.85,4900.0,71.2,21.9,3.112239e+07,27.6,0.70,0.0,0.0,0.0,5.1,0.0,1.8,0.0,93.10
4,Afghanistan,AF,"Muslim 99.7% (Sunni 84.7 - 89.7%, Shia 10 - 15...",34.2,4.43,2000.0,37.3,26.9,4.012155e+07,20.0,2.22,0.0,0.0,0.0,99.7,0.0,0.0,0.0,0.30


In [8]:
allowed_country_codes = [
    "AF", "AL", "AG", "AN", "AO", "AC", "AR", "AM", "AS", "AU", "AJ",
    "BF", "BA", "BG", "BB", "BO", "BE", "BH", "BN", "BT", "BL", "BK",
    "BC", "BR", "BX", "BU", "UV", "BM", "BY",
    "CB", "CM", "CA", "CV", "CT", "CD", "CI", "CH", "CO", "CN", "CG",
    "CF", "CS", "IV", "HR", "CU", "CY", "EZ",
    "DA", "DJ", "DO", "DR",
    "EC", "EG", "ES", "EK", "ER", "EN", "ET",
    "FJ", "FI", "FR",
    "GB", "GA", "GG", "GM", "GH", "GR", "GJ", "GT", "GV", "PU", "GY",
    "HA", "HO", "HU",
    "IC", "IN", "ID", "IR", "IZ", "EI", "IS", "IT",
    "JM", "JA", "JO",
    "KZ", "KE", "KR", "KN", "KS", "KV", "KU", "KG",
    "LA", "LG", "LE", "LT", "LI", "LY", "LS", "LH", "LU",
    "MK", "MA", "MI", "MY", "MV", "ML", "MT", "RM", "MR", "MP", "MX", "FM", "MD", "MN", "MG", "MJ", "MO", "MZ",
    "WA", "NR", "NP", "NL", "NZ", "NU", "NG", "NI", "NO",
    "MU",
    "PK", "PS", "PM", "PP", "PA", "PE", "RP", "PL", "PO",
    "QA",
    "RO", "RS", "RW",
    "SC", "ST", "VC",
    "WS", "SM", "TP", "SA", "SG", "RI", "SE", "SL", "SN",
    "LO", "SI", "BP", "SO", "SF", "OD", "SP", "CE", "SU", "NS",
    "WZ", "SW", "SZ", "SY",
    "TI", "TZ", "TH", "TT", "TO", "TN", "TD", "TS", "TU", "TX", "TV",
    "UG", "UP",
    "AE", "UK", "US", "UY", "UZ",
    "NH", "VT", "VE", "VM",
    "YM",
    "ZA", "ZI"
]


In [20]:
allowed_codes = pd.DataFrame({
    "Country Code": allowed_country_codes
})

df_filtered = pd.merge(allowed_codes, df, on="Country Code", how="inner")
df_filtered.head()

,Country Code,Country,Religions,Birth rate,Total fertility rate,GDP per capita,Literacy rate,Urban population,Population,Median age,Population growth rate,Protestant,Catholic,Orthodox,Muslim,Jewish,Other Christian,None,Other
0,AF,Afghanistan,"Muslim 99.7% (Sunni 84.7 - 89.7%, Shia 10 - 15...",34.2,4.43,2000.0,37.3,26.9,40121552.0,20.0,2.22,0.0,0.0,0.0,99.7,0.0,0.0,0.0,0.3
1,AL,Albania,"Muslim 56.7%, Roman Catholic 10%, Orthodox 6.8...",12.3,1.55,18200.0,98.4,64.6,3107100.0,36.3,0.16,0.0,10.0,6.8,56.7,0.0,0.0,0.0,26.5
2,AG,Algeria,"Muslim (official; predominantly Sunni) 99%, ot...",20.2,2.94,15200.0,81.4,75.3,47022473.0,29.1,1.54,0.0,0.0,0.0,99.6,0.2,0.2,0.0,0.0
3,AN,Andorra,"Christian (predominantly Roman Catholic) 89.5,...",6.9,1.47,64600.0,100.0,87.8,85370.0,48.8,0.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5
4,AO,Angola,"Roman Catholic 41.1%, Protestant 38.1%, other ...",41.1,5.70,7200.0,71.1,68.7,37202061.0,16.3,3.33,38.1,41.1,0.0,0.0,0.0,0.0,12.3,8.6


In [10]:
df_filtered.describe()

,Birth rate,Total fertility rate,GDP per capita,Literacy rate,Urban population,Population,Median age,Population growth rate,Protestant,Catholic,Orthodox,Muslim,Jewish,Other Christian,None,Other
count,148.000000,148.000000,148.000000,148.00000,148.000000,1.480000e+02,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,19.138514,2.565203,20405.405405,86.12500,58.162838,4.735440e+07,30.318243,1.243784,12.743863,18.912782,5.736261,27.657489,0.669313,10.300563,6.453041,14.075689
std,9.334535,1.157272,20675.440700,17.22181,21.807944,1.674444e+08,9.011097,0.893456,19.651774,26.421953,19.237565,37.431616,6.130968,21.517641,13.534347,21.737109
min,6.000000,1.120000,800.000000,26.80000,13.700000,2.186400e+04,15.200000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.500000,1.700000,5475.000000,79.75000,40.575000,2.759792e+06,22.075000,0.575000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.387500
50%,17.050000,2.150000,14200.000000,94.75000,59.250000,1.012670e+07,30.150000,0.970000,0.000000,3.600000,0.000000,4.900000,0.000000,1.000000,0.050000,6.000000
75%,25.725000,3.235000,29125.000000,98.45000,75.300000,3.485728e+07,36.350000,1.920000,20.250000,33.100000,0.000000,58.400000,0.000000,7.875000,6.400000,14.325000
max,46.600000,6.640000,127500.000000,100.00000,100.000000,1.416043e+09,48.800000,3.660000,79.300000,97.600000,90.100000,100.000000,73.500000,97.500000,86.300000,97.600000


In [11]:
df_filtered.to_csv("factbook_all_countries_cleaned.csv", index=False)

In [12]:
df_filtered["log_GDP_per_capita"] = np.log(df_filtered["GDP per capita"])
df_filtered["log_Population"] = np.log(df_filtered["Population"])

predictor_cols = ["log_GDP_per_capita", "Literacy rate", "Urban population", "Median age", "Population growth rate",
                  "Muslim", "Catholic", "Protestant", "Orthodox", "Jewish", "Other", "None", "Other Christian"]

scaler = StandardScaler()
df_filtered_std = df_filtered.copy()
df_filtered_std[predictor_cols] = scaler.fit_transform(df_filtered[predictor_cols])

df_filtered_std.head()

,Country Code,Country,Religions,Birth rate,Total fertility rate,GDP per capita,Literacy rate,Urban population,Population,Median age,...,Protestant,Catholic,Orthodox,Muslim,Jewish,Other Christian,None,Other,log_GDP_per_capita,log_Population
0,AF,Afghanistan,"Muslim 99.7% (Sunni 84.7 - 89.7%, Shia 10 - 15...",34.2,4.43,2000.0,-2.844695,-1.438420,40121552.0,-1.148948,...,-0.650686,-0.718229,-0.299193,1.931179,-0.109540,-0.480329,-0.478409,-0.635893,-1.656636,17.507424
1,AL,Albania,"Muslim 56.7%, Roman Catholic 10%, Orthodox 6.8...",12.3,1.55,18200.0,0.715179,0.296177,3107100.0,0.666075,...,-0.650686,-0.338470,0.055483,0.778516,-0.109540,-0.480329,-0.478409,0.573512,0.364515,14.949200
2,AG,Algeria,"Muslim (official; predominantly Sunni) 99%, ot...",20.2,2.94,15200.0,-0.275293,0.788490,47022473.0,-0.135653,...,-0.650686,-0.718229,-0.299193,1.928498,-0.076808,-0.471002,-0.478409,-0.649741,0.199652,17.666136
3,AN,Andorra,"Christian (predominantly Roman Catholic) 89.5,...",6.9,1.47,64600.0,0.808400,1.363622,85370.0,2.057964,...,-0.650686,-0.718229,-0.299193,-0.741389,-0.109540,-0.480329,-0.478409,-0.165056,1.523963,11.354750
4,AO,Angola,"Roman Catholic 41.1%, Protestant 38.1%, other ...",41.1,5.70,7200.0,-0.875403,0.484821,37202061.0,-1.560947,...,1.294653,0.842578,-0.299193,-0.741389,-0.109540,-0.480329,0.433476,-0.252760,-0.484245,17.431875


## Basic Correlation Matrix

In [24]:
df_filtered[['Total fertility rate', 'Muslim', 'Catholic', 'Protestant', 'None', 'GDP per capita', 'Literacy rate', 'Urban population', 'Population growth rate']].corr()["Total fertility rate"]


Total fertility rate      1.000000
Muslim                    0.229706
Catholic                 -0.077467
Protestant                0.106688
None                     -0.178103
GDP per capita           -0.566590
Literacy rate            -0.774585
Urban population         -0.446375
Population growth rate    0.912119
Name: Total fertility rate, dtype: float64

A basic correlation analysis shows that an increase in Islam and Protestantism is correlated with an *increase* in fertility rate, while an increase in Catholicism and Religious Nones is correlated with a *decline* in fertility rates.

However, this simply shows correlation. A regression analysis is required to check if there is any causative connection between the two.

## Regression Analysis

In [14]:
predictors = [
    "Muslim", "Catholic", "Protestant",
    "GDP per capita", "Literacy rate", "Urban population",
    "Population", "Median age", "Population growth rate"
]

In [15]:

raw_data = df_filtered.dropna(subset=["Total fertility rate"] + predictors)

y_raw = raw_data["Total fertility rate"]
X_raw = raw_data[predictors]
X_raw = sm.add_constant(X_raw)

model_raw = sm.OLS(y_raw, X_raw).fit()

print("----- Regression on Raw (Filtered) Data -----")
print(model_raw.summary())


----- Regression on Raw (Filtered) Data -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.896
Model:                              OLS   Adj. R-squared:                  0.889
Method:                   Least Squares   F-statistic:                     131.5
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           3.54e-63
Time:                          20:14:48   Log-Likelihood:                -63.926
No. Observations:                   148   AIC:                             147.9
Df Residuals:                       138   BIC:                             177.8
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

Initial regression analysis shows no connection between religion and fertility. While the P values are >0.05 the 95% confidence interval spans zero, so the true effect might be slightly negative, slightly positive, or zero. It's statistically indistinguishable from no effect.

In [16]:
norm_data = df_filtered_std.dropna(subset=["Total fertility rate"] + predictors)

y_norm = norm_data["Total fertility rate"]
X_norm = norm_data[predictors]
X_norm = sm.add_constant(X_norm)

model_norm = sm.OLS(y_norm, X_norm).fit()

print("\n----- Regression on Normalized Data -----")
print(model_norm.summary())



----- Regression on Normalized Data -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.896
Model:                              OLS   Adj. R-squared:                  0.889
Method:                   Least Squares   F-statistic:                     131.5
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           3.54e-63
Time:                          20:14:48   Log-Likelihood:                -63.926
No. Observations:                   148   AIC:                             147.9
Df Residuals:                       138   BIC:                             177.8
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

The results hold the same for normalised data.

### Multicollinearity Analysis

As different religious %\'s interact with one another, a final set of analyses are performed isolating the religions to investigate their individual impact.

In [17]:
# Define a base list of control variables
controls = ["GDP per capita", "Literacy rate", "Urban population", "Population", "Median age", "Population growth rate"]

def model_by_religion(religion):
    predictors_religion = [religion] + controls
    X_religion = df_filtered.dropna(subset=["Total fertility rate"] + predictors_religion)[predictors_religion]
    y_religion = df_filtered.dropna(subset=["Total fertility rate"] + predictors_religion)["Total fertility rate"]

    X_religion = sm.add_constant(X_religion)
    model_religion = sm.OLS(y_religion, X_religion).fit()
    print(f"----- Model with {religion} share only -----")
    print(model_religion.summary())

model_by_religion("Muslim")

----- Model with Muslim share only -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.896
Model:                              OLS   Adj. R-squared:                  0.890
Method:                   Least Squares   F-statistic:                     171.4
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           2.16e-65
Time:                          20:14:49   Log-Likelihood:                -63.977
No. Observations:                   148   AIC:                             144.0
Df Residuals:                       140   BIC:                             167.9
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [18]:
model_by_religion("Catholic")

----- Model with Catholic share only -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.895
Model:                              OLS   Adj. R-squared:                  0.890
Method:                   Least Squares   F-statistic:                     171.2
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           2.36e-65
Time:                          20:14:49   Log-Likelihood:                -64.070
No. Observations:                   148   AIC:                             144.1
Df Residuals:                       140   BIC:                             168.1
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

In [19]:
model_by_religion("Protestant")

----- Model with Protestant share only -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.895
Model:                              OLS   Adj. R-squared:                  0.890
Method:                   Least Squares   F-statistic:                     171.3
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           2.22e-65
Time:                          20:14:49   Log-Likelihood:                -64.005
No. Observations:                   148   AIC:                             144.0
Df Residuals:                       140   BIC:                             168.0
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [21]:
model_by_religion("None")

----- Model with None share only -----
                             OLS Regression Results                             
Dep. Variable:     Total fertility rate   R-squared:                       0.896
Model:                              OLS   Adj. R-squared:                  0.890
Method:                   Least Squares   F-statistic:                     171.5
Date:                  Sun, 23 Mar 2025   Prob (F-statistic):           2.09e-65
Time:                          20:27:04   Log-Likelihood:                -63.944
No. Observations:                   148   AIC:                             143.9
Df Residuals:                       140   BIC:                             167.9
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

No effect is found when isolated either.

The, surprising, conclusion from the dataset is that religion has no statistically significant effect on fertility rates once one accounts for socio-economic factors.